In [7]:
import numpy as np
import sep

ModuleNotFoundError: No module named 'numpy'

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as mlp
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

ModuleNotFoundError: No module named 'astropy'

**Image Display**

via SEP instruction linked, define the F105W band image of the Hubble Ultra Deep Field from the FITS file
defined ->

In [1]:
#defining file name
fdata_F105W = "image.fits"
#read data
hdu_1 = fits.open(fdata_image)
#get image data
data = hdu_1[0].data

NameError: name 'fits' is not defined

Displaying the image ->

In [ ]:
#show the image
m, s = np.mean(data), np.std(data)
mlp.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
mlp.colorbar();
mlp.savefig('SEP_Image.png',bbox_inches="tight",dpi=600)

**Background Subtraction**

Using SEP to substract background and find sources

In [ ]:
#measure spatially varying background on the image
bkg = sep.Background(data)

NameError: name 'data_2' is not defined

In [ ]:
#getting "global" mean and noise of image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
# bkg_image = np.array(bkg) # equivalent to above

In [ ]:
# show the background
mlp.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
mlp.colorbar();
mlp.savefig('Background_Image.png',bbox_inches="tight",dpi=600)

NameError: name 'mlp' is not defined

In [ ]:
#Subtracting the background
data_sub = data - bkg

**Object Detection**

With the removal of background noise, we can now run object detection on the data, and according to the SEP tutorial, the detection threshold should be at the constant value 1.5σ where σis the global background RMS (root mean square)

In [ ]:
#define 'objects'
objects =sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
# how many objects were detected
len(objects)

Now that we have the number of objects, we can check the location of detected objects ->

In [ ]:
#How to circle objects via matplotlib Ellipses
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = mlp.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('Detection_Image.png',bbox_inches="tight",dpi=600)

In [ ]:
# available fields
objects.dtype.names

**Aperture photometry**

Finally, we’ll perform simple circular aperture photometry with a 3 pixel radius at the locations of the objects:

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))